In [ ]:
%%capture
!pip install scikit-learn
!pip install fpdf

In [42]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        # Arial bold 15
        self.set_font('Arial', 'B', 15)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        self.set_x((210 - w) / 2)
        # Colors of frame, background and text
        self.set_draw_color(0, 0, 0)
        self.set_fill_color(220, 220, 250)
        self.set_text_color(0, 0, 0)
        # Thickness of frame (1 mm)
        self.set_line_width(1)
        # Title
        self.cell(w, 9, title, 1, 1, 'C', 1)
        # Line break
        self.ln(10)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        # Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

    def chapter_title(self, num, label):
        # Arial 12
        self.set_font('Arial', '', 12)
        # Background color
        self.set_fill_color(200, 220, 255)
        # Title
        self.cell(0, 6, 'Model %d : %s' % (num, label), 0, 1, 'L', 1)
        # Line break
        self.ln(4)

    def chapter_body(self, report):
        self.set_font('Times', '', 12)
        # Output justified text
        self.multi_cell(0, 5, f"Accuracy: {report['Accuracy']}")
        # Line break
        self.ln()

        self.multi_cell(0, 5, f"Precision: {report['Precision']}")
        # Line break
        self.ln()
        
        self.multi_cell(0, 5, f"Recall: {report['Recall']}")
        # Line break
        self.ln()
        # Mention in italics
        self.set_font('', 'I')
        

    def print_chapter(self, num, report):
        self.add_page()
        self.chapter_title(num, report["Model Name"])
        self.chapter_body(report)

In [43]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score

class ModelEvaluator:
    def __init__(self, model_name, model):
        self.model_name = model_name
        self.model = model

    def report(self, X_test, y_test):
        
        y_pred = self.model.predict(X_test)
  
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)

        return {
            'Model Name': self.model_name,
            'Accuracy': round(accuracy,3),
            'Precision': round(precision,3),
            'Recall': round(recall,3)
        }

class ModelDummy():
    def __init__(self, name):
        self.name = name

    def predict(self,X_test):
        return np.random.randint(2, size=X_test.shape[0])

In [44]:
X_test = np.array([1,0,1,0,0,1])
y_test = np.array([1,1,0,0,1,1])


model1 = ModelDummy(name="model abc v1")
evaluator1 = ModelEvaluator("model abc v1", model1)
report1 = evaluator1.report(X_test, y_test)

model2 = ModelDummy(name="model xyz v2")
evaluator2 = ModelEvaluator("model xyz v2", model2)
report2 = evaluator2.report(X_test, y_test)

pdf = PDF()

title = 'Performance Report V1.0'
pdf.set_title('Performance Report V1.0')

pdf.print_chapter(1, report1)
pdf.print_chapter(2, report2)

pdf.output('Performance Report V1.0.pdf', 'F')

''